# ANN Notebook

#### *Author: Kunyu He*
#### *University of Chicago, CAPP'20*

In [18]:
import pandas as pd
import numpy as np
import keras

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix

### Load Data

In [2]:
churn = pd.read_csv("Churn_Modelling.csv")
churn.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


The bank witnessed high churn (customers leave bank) rate, and wants to lower the exiting rate. We are predicting which customer is leaving the bank.

### Data Cleaning

In [3]:
churn.shape

(10000, 14)

In [4]:
churn.isnull().sum(axis=0)

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

No value missing.

### Feature Engineering

In [5]:
X = churn.iloc[:, 3:13].values
X.shape

(10000, 10)

In [6]:
y = churn.Exited.values.astype(float)
y.shape

(10000,)

In [7]:
X[:, 2] = LabelEncoder().fit_transform(X[:, 2])

dummies = OneHotEncoder(categories="auto").fit_transform(X[:, [1]]).toarray()
X = np.delete(X, 1, 1)
X = np.column_stack((dummies[:, 1:], X)).astype(float) # drop one category

X.shape

(10000, 11)

### Training and Test Set Split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=123)

### Feature Scaling

In [9]:
sc_X = StandardScaler()

X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

### Model Training

In [14]:
ann = Sequential()

Tip: number of neurons in the hidden layers can be set to the average numbers of neurons in the input and output layers as an initial guess.

In [15]:
ann.add(Dense(512, activation="relu", input_shape=(11,)))
ann.add(Dense(512, activation="relu"))
ann.add(Dense(1, activation="sigmoid"))

In [16]:
ann.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [17]:
ann.fit(X_train, y_train, batch_size=10, epochs=50)

Epoch 1/50
8000/8000 [==============================] - 6s 764us/step - loss: 0.3913 - acc: 0.8387
Epoch 2/50
8000/8000 [==============================] - 6s 703us/step - loss: 0.3542 - acc: 0.8552
Epoch 3/50
8000/8000 [==============================] - 6s 704us/step - loss: 0.3442 - acc: 0.8612
Epoch 4/50
8000/8000 [==============================] - 6s 705us/step - loss: 0.3402 - acc: 0.8607
Epoch 5/50
8000/8000 [==============================] - 6s 706us/step - loss: 0.3338 - acc: 0.8631
Epoch 6/50
8000/8000 [==============================] - 6s 704us/step - loss: 0.3291 - acc: 0.8646
Epoch 7/50
8000/8000 [==============================] - 6s 708us/step - loss: 0.3263 - acc: 0.8660
Epoch 8/50
8000/8000 [==============================] - 6s 706us/step - loss: 0.3227 - acc: 0.8675
Epoch 9/50
8000/8000 [==============================] - 6s 706us/step - loss: 0.3148 - acc: 0.8675
Epoch 10/50
8000/8000 [==============================] - 6s 706us/step - loss: 0.3123 - acc: 0.8714
Epoch 11/

### Model Evaluation

In [26]:
confusion_matrix(y_test, ann.predict(X_test) > 0.5)

array([[1449,  137],
       [ 221,  193]], dtype=int64)

Obviously, our model is over-fitting the training set. Try a less complex one with less neurons in the hidden layers.

In [28]:
ann6 = Sequential()
ann6.add(Dense(6, activation="relu", input_shape=(11,)))
ann6.add(Dense(6, activation="relu"))
ann6.add(Dense(1, activation="sigmoid"))
ann6.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

ann6.fit(X_train, y_train, batch_size=10, epochs=20)

Epoch 1/20
8000/8000 [==============================] - 4s 453us/step - loss: 0.5060 - acc: 0.7967
Epoch 2/20
8000/8000 [==============================] - 3s 410us/step - loss: 0.4389 - acc: 0.7995
Epoch 3/20
8000/8000 [==============================] - 3s 411us/step - loss: 0.4239 - acc: 0.8127
Epoch 4/20
8000/8000 [==============================] - 3s 412us/step - loss: 0.4147 - acc: 0.8166
Epoch 5/20
8000/8000 [==============================] - 3s 423us/step - loss: 0.4051 - acc: 0.8202
Epoch 6/20
8000/8000 [==============================] - 3s 411us/step - loss: 0.3952 - acc: 0.8286
Epoch 7/20
8000/8000 [==============================] - 3s 417us/step - loss: 0.3856 - acc: 0.8399
Epoch 8/20
8000/8000 [==============================] - 3s 414us/step - loss: 0.3751 - acc: 0.8439
Epoch 9/20
8000/8000 [==============================] - 3s 421us/step - loss: 0.3665 - acc: 0.8501
Epoch 10/20
8000/8000 [==============================] - 3s 419us/step - loss: 0.3598 - acc: 0.8505
Epoch 11/

In [29]:
confusion_matrix(y_test, ann6.predict(X_test) > 0.5)

array([[1527,   59],
       [ 217,  197]], dtype=int64)

No over-fitting.